In [ ]:
import psycopg2
import csv

# Подключение к базе данных
conn = psycopg2.connect(
    host="localhost",
    database="phonebookdb",
    user="postgres",
    password="12345678",
    port=5432
)
cur = conn.cursor()

# Создание таблицы, если не существует
cur.execute("""
    CREATE TABLE IF NOT EXISTS numbers (
        name TEXT,
        surname TEXT,
        phone_number TEXT PRIMARY KEY
    )
""")
conn.commit()

# SQL-запросы
sqlIN = 'INSERT INTO numbers(name, surname, phone_number) VALUES (%s, %s, %s) RETURNING *'
sqlDEL = 'DELETE FROM numbers WHERE phone_number = %s'
sqlUP_name = 'UPDATE numbers SET name = %s WHERE phone_number = %s'
sqlUP_surname = 'UPDATE numbers SET surname = %s WHERE phone_number = %s'
sqlUP_phone = 'UPDATE numbers SET phone_number = %s WHERE name = %s AND surname = %s'
sqlFIND_phone = 'SELECT * FROM numbers WHERE phone_number = %s'
sqlFIND_name = 'SELECT * FROM numbers WHERE name = %s AND surname = %s'
sqlALL = 'SELECT * FROM numbers'

# CSV-загрузка
def read_csv(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [
            tuple(row) for i, row in enumerate(csv.reader(f))
            if i > 0 and len(row) == 3
        ]

# Меню
print('to Insert manually type 1')
print('to insert from csv type 2')
print('to delete record type 3')
print('to find the number type 4')
print('to update type 5')
print('show all type 6')

choice = int(input("Your choice: "))

if choice == 1:
    name = input("Name: ")
    surname = input("Surname: ")
    phone_number = input("Phone number (11 digits): ")

    if len(phone_number) == 11 and phone_number.isdigit():
        cur.execute(sqlFIND_name, (name, surname))
        if cur.fetchone():
            cur.execute(sqlUP_phone, (phone_number, name, surname))
        else:
            cur.execute(sqlFIND_phone, (phone_number,))
            if cur.fetchone():
                cur.execute(sqlUP_name, (name, phone_number))
                cur.execute(sqlUP_surname, (surname, phone_number))
            else:
                cur.execute(sqlIN, (name, surname, phone_number))
        conn.commit()
    else:
        print("Invalid number")

elif choice == 2:
    data = read_csv('LegendsPhoneBook02.csv')
    cur.executemany(sqlIN, data)
    conn.commit()
    print("CSV data inserted.")

elif choice == 3:
    phone_number = input("Type number: ")
    cur.execute(sqlFIND_phone, (phone_number,))
    if cur.fetchone():
        cur.execute(sqlDEL, (phone_number,))
        conn.commit()
        print("Deleted")
    else:
        print("Invalid number")

elif choice == 4:
    method = input("Find by name - 1 or phone - 2: ")
    if method == "1":
        name = input("Name: ")
        surname = input("Surname: ")
        cur.execute(sqlFIND_name, (name, surname))
    else:
        phone_number = input("Phone number: ")
        cur.execute(sqlFIND_phone, (phone_number,))
    result = cur.fetchone()
    print(result if result else "Not found")

elif choice == 5:
    phone_number = input("Old phone number: ")
    cur.execute(sqlFIND_phone, (phone_number,))
    if cur.fetchone():
        name = input("New name (leave empty to skip): ")
        surname = input("New surname (leave empty to skip): ")
        new_phone_number = input("New phone number (11 digits, leave empty to skip): ")

        if name:
            cur.execute(sqlUP_name, (name, phone_number))
        if surname:
            cur.execute(sqlUP_surname, (surname, phone_number))
        if new_phone_number and len(new_phone_number) == 11 and new_phone_number.isdigit():
            cur.execute(sqlUP_phone, (new_phone_number, name or 'name', surname or 'surname'))
        conn.commit()
        print("Updated")
    else:
        print("Invalid old number")

elif choice == 6:
    cur.execute(sqlALL)
    for row in cur.fetchall():
        print(row)

else:
    print("Invalid choice")

cur.close()
conn.close()


to Insert manually type 1
to insert from csv type 2
to delete record type 3
to find the number type 4
to update type 5
show all type 6
